 **Creating function to import json file**
 
Importing image id and annotations from json file



In [ ]:
import json
def json_to_dict(file):
    with open(file) as f:
        json_data = json.load(f)
        images = json_data['images']
        annotations = json_data['annotations']
        return(images,annotations)


**Storing imported images and annotations**

In [ ]:
images, annotations = json_to_dict('./train.json')

**Creating list of annotion and images id**

In [ ]:
list_annot = []
for annotation in annotations:
    annot = { "caption" : annotation["caption"], 
            "image_id" : annotation["image_id"],
             "id":annotation["id"]
            }
    list_annot.append(annot)

**Function to convert into panda dataframe**

In [ ]:
import pandas as pd
def convert_to_df(dict_):
    return pd.DataFrame.from_dict(dict_)

**Creating dataframe from the list**

Calculating length of caption

In [ ]:

annot_df = convert_to_df(list_annot)
annot_df['caption_len'] = annot_df['caption'].apply(len)
annot_df.head()


,caption,image_id,id,caption_len
0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS T...,0,0,64
1,A green and white plastic condiment bottle con...,0,1,67
2,Quality issues are too severe to recognize vis...,0,2,58
3,A bottle of spices in a plastic container layi...,0,3,62
4,some basil leaves in a container on a counter,0,4,45


In [ ]:
len(annot_df)

117155

**Using Groupby method to group image id**

In [ ]:
grouped_single=annot_df.groupby('image_id')['caption_len'].apply(max).reset_index().rename(columns={'image_id':'image_id','caption_len' : 'caption_len'})


grouped_single.head()


,image_id,caption_len
0,0,67
1,1,104
2,2,98
3,3,132
4,4,63


In [ ]:
len(grouped_single)

23431

**Merging dataframe**

In [ ]:
dfinal = grouped_single.merge(annot_df,on = ["image_id", "caption_len"], how="left")
dfinal.head()
 

,image_id,caption_len,caption,id
0,0,67,A green and white plastic condiment bottle con...,1
1,1,104,A kitchen counter the various items on top inc...,7
2,2,98,A can of crushed tomatoes are on a brown surfa...,10
3,3,132,A screenshot of Spotify page on a cell phone s...,19
4,4,63,A box containing information about a solar gar...,21


**Removing duplicates image id**

In [ ]:
dfinal = dfinal.drop_duplicates(subset=['image_id']).drop("caption_len",1)
dfinal.head()

,image_id,caption,id
0,0,A green and white plastic condiment bottle con...,1
1,1,A kitchen counter the various items on top inc...,7
2,2,A can of crushed tomatoes are on a brown surfa...,10
3,3,A screenshot of Spotify page on a cell phone s...,19
4,4,A box containing information about a solar gar...,21


In [ ]:
dfinal=dfinal.reset_index()

In [ ]:
len(dfinal) == len(images)

True

**Converting dataframe into dictionary**

In [ ]:
annot_dict= dfinal.to_dict('r')
annot_dict[7740]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


{'caption': 'Box of instant pudding with a mug on a plate in the background overflowing with some substance.',
 'id': 38702,
 'image_id': 7740,
 'index': 9316}

In [ ]:
images[0]

{'file_name': 'VizWiz_train_00000000.jpg',
 'id': 0,
 'text_detected': True,
 'vizwiz_url': 'https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000000.jpg'}

In [ ]:
len(annot_dict)

23431

In [ ]:
json_final = {
    "images": images,
    "annotations":annot_dict
}

In [ ]:

with open('./captions_vizwiz2020.json', 'w') as json_file:
    json.dump(json_final, json_file)

In [ ]:
#Saving top 200

In [ ]:
 len(images[0:200]) == len(dfinal.head(200))

True

In [ ]:
json_final_200 = {
    "images": images[0:200],
    "annotations":dfinal.head(200).to_dict('r')
}

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


**Dumping final 200 captions and image id**

In [ ]:
with open('./captions_vizwiz2020_top200.json', 'w') as json_file:
    json.dump(json_final_200, json_file)